## Methodology for ANDE Report on Donor Contributions to SGBs   

## 1. Preparing the data

This notebook is intended to demonstrate the methodology used by the Devex Analytics team to arrive at total figures for funding channeled by 20 different donor institutions to the "thematic area" of *entrepreneurship and small-and-growing businesses*. 

The methodology uses data from the International Aid Transparency Initiative (IATI). The final figures in the Devex report summed the figures from this methodology, as well as other figures found through additional desk research. Thus this methodology does not represent a complete data collection process. However, the code and descriptions here illustrate well the process followed for IATI data, and the human decisions made about how to analyse that data. 

Note that some donors are consistent, high-quality contributors of IATI data, while others contribute less frequently or less detailed data, and others do not contribute data at all to IATI. For donors that do *not* consistently contribute quality data to IATI, additional collection, processing, and analysis will be required.

In [1]:
# Importing the needed libraries
import pandas as pd
import numpy as np

Let's import the data. First - there are generally 56 columns of data in a IATI activities datafile. We only want some of them:

In [2]:
iati_fields = ['iati-identifier','reporting-org','default-language', 'title','description','start-planned','end-planned',\
               'start-actual', 'end-actual','recipient-country-code','recipient-country', 'recipient-country-percentage',\
               'sector','sector-code', 'sector-percentage','sector-vocabulary','sector-vocabulary-code', 'default-currency',\
               'total-Commitment','total-Disbursement','total-Expenditure']
date_fields = ['start-planned','end-planned','start-actual','end-actual']

We'll use this to only import 21 columns relevant to our analysis. We'll specify which fields are dates so we can upload them in the useful *datetime* data type.

I've downloaded large csv files from donors who are good IATI contributors - the World Bank, DFID, Sida, the Gates Foundation, EuropeAid (Devco), Global Affairs Canada, the Global Fund, and the Ministry of Foreign Affairs of the Netherlands. In the future, we'll want to replace these calls to CSVs with a request to IATI's API. 

In [3]:
# NB! Replace the read_csv commands with IATI API query in the future
wbg_raw = pd.read_csv('WBG_IATI_Activities_20190315.csv', low_memory=False, usecols=iati_fields, parse_dates=date_fields)
dfid_raw = pd.read_csv('DFID_IATI_Activities_20190315.csv', low_memory=False, usecols=iati_fields, parse_dates=date_fields)
sida_raw = pd.read_csv('SIDA_IATI_Activities_20190315.csv', low_memory=False, usecols=iati_fields, parse_dates=date_fields)
bmgf_raw = pd.read_csv('BMGF_IATI_Activities_20190315.csv', low_memory=False, usecols=iati_fields, parse_dates=date_fields)
devco_raw = pd.read_csv('DEVCO_IATI_Activities_20190315.csv', low_memory=False, usecols=iati_fields, parse_dates=date_fields)
gac_raw = pd.read_csv('GAC_IATI_Activities_20190315.csv', low_memory=False, usecols=iati_fields, parse_dates=date_fields)
gf_raw = pd.read_csv('Global_Fund_IATI_Activities_20190315.csv', low_memory=False, usecols=iati_fields, parse_dates=date_fields)
MFANe_raw = pd.read_csv('MFA_Netherlands_IATI_Activities_20190315.csv', low_memory=False, usecols=iati_fields, parse_dates=date_fields)

dfs = [wbg_raw, dfid_raw, sida_raw, bmgf_raw, devco_raw, dfid_raw, gac_raw, gf_raw, MFANe_raw]

Next we put the data together in a single Pandas dataframe.

In [4]:
data = pd.concat(dfs, ignore_index=True, sort=False)

Now let's see what the data looks like by seeing how many rows of data each donor contributed to the data:

In [5]:
data.groupby(['reporting-org'])['iati-identifier'].count()

reporting-org
Bill and Melinda Gates Foundation                               9579
Department for International Development                       36706
European Commission - Development and Cooperation-EuropeAid    67158
Foreign Affairs, Trade and Development Canada (DFATD)           4476
Ministry of Foreign Affairs (DGIS)                              7730
Sweden                                                         85601
The Global Fund to Fight AIDS, Tuberculosis and Malaria         1100
World Bank                                                      3035
World Bank Group                                                  30
Name: iati-identifier, dtype: int64

In [6]:
data.shape

(215415, 21)

We have 21 columns and 215,415 rows of data. There are a few cosmetic changes to make: Let's check the 30 rows from the "World Bank Group" as opposed to the "World Bank", and let's change the names of the donors to the donor shortnames for readability.

In [7]:
# Rename donors 'reporting-org' field to their donor shortname
data.loc[data['reporting-org'] == 'Sweden', 'reporting-org'] = 'Sida'
data.loc[data['reporting-org'] == 'Department for International Development', 'reporting-org'] = 'DFID'
data.loc[data['reporting-org'] == 'European Commission - Development and Cooperation-EuropeAid', 'reporting-org'] = 'DEVCO'
data.loc[data['reporting-org'] == 'Foreign Affairs, Trade and Development Canada (DFATD)', 'reporting-org'] = 'GAC'
data.loc[data['reporting-org'] == 'Ministry of Foreign Affairs (DGIS)', 'reporting-org'] = 'MFA Netherlands'
data.loc[data['reporting-org'] == 'The Global Fund to Fight AIDS, Tuberculosis and Malaria', 'reporting-org'] = 'Global Fund'
data.loc[data['reporting-org'] == 'Bill and Melinda Gates Foundation', 'reporting-org'] = 'B&MGF'
# Let's check that it worked.
data.groupby(['reporting-org'])['iati-identifier'].count()

reporting-org
B&MGF                9579
DEVCO               67158
DFID                36706
GAC                  4476
Global Fund          1100
MFA Netherlands      7730
Sida                85601
World Bank           3035
World Bank Group       30
Name: iati-identifier, dtype: int64

It worked - much easier to read the results of the groupby() function by donor now.

Let's look at the differences between 'World Bank' and 'World Bank Group' rows now. The most important fields for our analysis will be the dates, countries, sectors, and funding commitments, so we'll look at them.

In particular, for this methodology to work well, we'll need rows that denote IATI activities which are tagged with OECD sector vocabulary tags. Since there are only 30 *World Bank Group*, we can quickly look at all the rows there.

In [8]:
wb_condition = data['reporting-org'] == 'World Bank'
wbg_condition = data['reporting-org'] == 'World Bank Group'
wb_rows = data[wb_condition].reset_index()
wbg_rows = data[wbg_condition].reset_index()

In [9]:
wbg_rows.loc[:, 'sector-code':'sector-vocabulary-code']

,sector-code,sector,sector-percentage,sector-vocabulary,sector-vocabulary-code
0,FY,NaN,100,NaN,WBSector
1,TH,NaN,100,NaN,WBSector
2,AC,NaN,100,NaN,WBSector
3,AI,NaN,100,NaN,WBSector
4,AL,NaN,100,NaN,WBSector
5,TY,NaN,100,NaN,WBSector
6,TH,NaN,100,NaN,WBSector
7,AA,NaN,100,NaN,WBSector
8,WW,NaN,100,NaN,WBSector
9,AL,NaN,100,NaN,WBSector


Looks like the first 15 rows have no sector data, which is of no use to us. The last 15 rows have only WBSector or WBTheme as a sector vocabulary, which is not terribly useful to us at this point. We can drop these 30 rows from our data.

In [10]:
data.drop(data[data['reporting-org'] == 'World Bank Group'].index, inplace=True)
data.groupby(['reporting-org'])['iati-identifier'].count()

reporting-org
B&MGF               9579
DEVCO              67158
DFID               36706
GAC                 4476
Global Fund         1100
MFA Netherlands     7730
Sida               85601
World Bank          3035
Name: iati-identifier, dtype: int64

Great. We will also want to remove any other rows that don't contain OECD as a sector vocabulary. 

Let's check how each donor has coded their sectors and sector vocabularies in their data by printing a quick sample of rows from each donor.   

In [11]:
reporting_orgs = ['B&MGF', 'DFID', 'DEVCO', 'GAC', 'MFA Netherlands', 'Sida', 'Global Fund', 'World Bank']

for org in reporting_orgs:
    condition = data['reporting-org'] == org
    rows = data[condition].reset_index()
    print(org + '\n')
    print(rows.loc[0:3, 'sector-code':'sector-vocabulary-code'])
    print('\n')

#sida_condition = data['reporting-org'] == 'Swedish International Development Agency (Sida)'
#sida_rows = data[sida_condition].reset_index()
#dfid_condition = data['reporting-org'] == 'Department for International Development'
#dfid_rows = data[dfid_condition].reset_index()

B&MGF

  sector-code                          sector sector-percentage  \
0       12250      Infectious disease control               100   
1       12250      Infectious disease control               100   
2       13030                 Family planning               100   
3       13040  STD control including HIV/AIDS               100   

  sector-vocabulary sector-vocabulary-code  
0               NaN                      1  
1               NaN                      1  
2               NaN                      1  
3               NaN                      1  


DFID

                           sector-code  \
0                    15160;15150;13020   
1                    15160;15150;13020   
2                    15160;15150;13020   
3  15170;15160;15153;15150;15110;15151   

                                              sector  sector-percentage  \
0  ;Democratic participation and civil society;Re...           30;30;40   
1  ;Democratic participation and civil society;Re...           

If we look at the sample rows for each donor, we see that all these donors use the IATI field *sector-vocabulary-code*. This field should be filled with 1 or 2 digit codes indicating what sector scheme (or schemes) are used to classify the sector of each IATI activity. The code for OECD sector vocabulary is '1'. 

It appears that some donors use the field incorrectly, and often insert the string 'DAC' to indicate use of the OECD sector scheme, rather than using '1'. However, all the donors use the field consistently, and either use '1' or 'DAC'. We could do a more thorough check that this is true, but for now we'll roll with the assumption that any row with an OECD sector tagged will also have either '1' or 'DAC' in the *sector-vocabulary-code*.

To extract the rows that contain activities with OECD sectors, we'll filter our data to any rows that contain either '1' or 'DAC' in that row's value for *sector-vocabulary-code*. 

## 2. Filtering for relevant OECD Sectors

Why are we interested in activities tagged with OECD DAC sectors?

For two reasons: first, the OECD DAC sector scheme is the most utilized by donors that contribute data to IATI. Nearly all major donors tag their activities with DAC sectors - so this sector scheme offers a means to filter all the donors' data together. 

Second, our methodology uses DAC sectors to approximate concepts relevant to our donor funding research. Our methodology takes a "concept of interest", such as "small and growing businesses and entrepreneurship", or "reproductive, neonatal, maternal, child, and adolescent health" that are NOT well-defined in the OECD sector scheme, or indeed in any sector scheme. We then pick the OECD sectors that best approximate this "concept of interest", and look at only the IATI data tagged with those "nearest-to-our-concept" sectors.

In this analysis the example topic of interest will be "*entrepreneurship and small-and-growing businesses*". There are several OECD sectors that approximate this idea, but we'll pick one: *32130 - Small and medium enterprises (SME) development*. Sectors with this tag have the text '32130' at some place in the field *sector-code*.

First let's remove any rows that don't contain a reference to the OECD sector vocabulary:

In [12]:
# Fill any NaN values in the sector-vocabulary-code field
data['sector-vocabulary-code'] = data['sector-vocabulary-code'].fillna(value='')
# Create two subframes, one filtered for sector-vocabulary-code containing 'DAC', one for '1', then concatenate them.
oecd_data = pd.concat([data[data['sector-vocabulary-code'].str.contains('DAC', na=False)], \
                  data[data['sector-vocabulary-code'].str.contains('1', na=False)]], \
                 ignore_index=True, sort=False)

In [13]:
oecd_data.shape

(148123, 21)

Great, now our *data* dataframe only contains rows that contain at least one OECD sector tag. We went from 215K rows to xxxK rows of data. Let's check that we still have a decent amount of data from all the donors: 

In [14]:
oecd_data.groupby(['reporting-org'])['iati-identifier'].count()

reporting-org
B&MGF               9579
DFID               36608
GAC                 4475
Global Fund         1100
MFA Netherlands     7730
Sida               85601
World Bank          3030
Name: iati-identifier, dtype: int64

In [15]:
sectors_of_interest = ['12110', '12181', '12182', '12191', '12220', '12230', '12240', '12250', '12261', \
                      '12262', '12263', '12281', '12310', '12320', '12330', '12340', '12350', '12382', \
                      '13010', '13020', '13030', '13040', '13081', '15170', '15180', '16063','93013']
sectors_of_interest2 = ['13010', '13020', '13030', '13040', '13081']
sector_data = pd.DataFrame()
for sector in sectors_of_interest2:
    temp = oecd_data[oecd_data['sector-code'].str.contains(sector, na=False)]
    sector_data = pd.concat([temp, sector_data], ignore_index=True, sort=False)

In [16]:
sector_data.shape

(9929, 21)

In [17]:
sector_data.groupby(['reporting-org'])['iati-identifier'].count()

reporting-org
B&MGF         2038
DFID          3250
GAC            647
Sida          3878
World Bank     116
Name: iati-identifier, dtype: int64

Now we can see that the amount of data is greatly reduced - nearly 2K rows where we started with 215K. 

We also see that some donors had *no* data related to DAC sector 32130 - only 4 of our original 8 sectors remain. This is problematic, but we'll address it later.

We can even have a quick, dirty look at the sums of funding commitments to this OECD sector from each of the three donors - though this comparison isn't necessarily valid yet, for reasons we'll see...

In [18]:
sector_data['commitment-in-billions'] = sector_data.apply(lambda row: row['total-Commitment'] / 1000000000, axis=1)
sector_data.groupby(['reporting-org'])['commitment-in-billions'].sum()

reporting-org
B&MGF          7.397211
DFID          25.128330
GAC           11.243735
Sida           2.478802
World Bank    13.854913
Name: commitment-in-billions, dtype: float64

Keep in mind that the different donors may use different currencies - for instance, DFID uses GBP, Sida uses SEK, and the World Bank uses USD.

However, we are comparing totals over different time periods, which isn't valid. Additionally, some activities' funding commitments are only *partially* commited to the sector of interest. We need to restrict the data to a time window of analysis, and account for any partial funding commitments.

Last thing - to keep the data clean, we'll delete the *commitment-in-billions* field we just created.

In [19]:
sector_data = sector_data.drop(['commitment-in-billions'], axis=1).reset_index()
sector_data = sector_data.drop(['index'], axis=1)

## 3. Choosing a timeframe for analysis and scaling the commitments

We now have donor commitments to our 'concept of interest' as approximated by one or more OECD DAC sectors. However, the funding totals above sum *all the funding* from each donor over the *entire history* of the donors' IATI activities. 

What if we only want to compare donors' commitments to our 'concept of interest' over a 'time period of interest'? For instance, what if we want donor commitments in 2017?

IATI activities' lives are frequently longer than one year, and may begin in the middle of one year, and end in the middle of another. Thus, we need to determine the 'average commitment per year' of each activity, and determine whether or not they were active (and for how long, in years) during our 'time period of interest'. 

To this end, we have four date fields in the data: the planned start and end dates of each activity, and the actual start and end dates of each activity. However there are lots of missing values. 

In [20]:
sector_data.loc[0:5, 'start-planned':'end-actual']

,start-planned,end-planned,start-actual,end-actual
0,2019-05-01,2026-05-30 00:00:00,2018-10-26,NaT
1,2019-05-01,2024-05-30 00:00:00,2019-05-01,NaT
2,2019-05-01,2026-05-30 00:00:00,2019-05-01,NaT
3,2012-03-01,2017-03-31 00:00:00,2013-07-11,2017-05-29
4,2012-03-01,2017-03-31 00:00:00,2012-03-01,2017-03-31
5,2013-07-11,2017-03-31 00:00:00,2013-07-11,2017-03-31


To calculate the length in years of each activity, we'll need to check that data exists in these fields for these fields. To do so, we'll do a quick IF - THEN analysis of each activity, and calculate the difference in years accordingly:

        IF (start-actual exists AND end-actual exists)
        THEN difference = (end-actual - start-actual), then divide by 365.25 to arrive at years diff
        ELIF (start-actual exists AND end-planned exists)
        THEN difference = (end-planned - start-actual), then divide by 365.25 to arrive at years diff
        ELIF (start-planned exists and end-planned exists)
        THEN difference = (end-planned - start-planned), then divide by 365.25 to arrive at years diff
        ELSE leave difference blank for now
        
To apply this logic to our data, we'll write  function to determine which IF scenario applies, calculate the difference in days, and then convert to a difference in years.

In [21]:
def dateDiff(dataFrame):
    
    for i in dataFrame.index:
        
        # both end-actual and start-actual are not null - i.e. they are dates
        if not pd.isnull(dataFrame.at[i,'end-actual']) and not pd.isnull(dataFrame.at[i,'start-actual']):
            dataFrame.at[i,'date-diff-days'] = dataFrame.at[i,'end-actual'] - dataFrame.at[i,'start-actual']
            # if less than 365.25 days, round the duration in years to 1
            if dataFrame.at[i,'date-diff-days'].days / 365.25 > 1:
                dataFrame.at[i,'date-diff-years'] = dataFrame.at[i,'date-diff-days'].days / 365.25
            else:
                dataFrame.at[i,'date-diff-years'] = 1
        
        # both end-planned and start-actual are not null - i.e. they are dates
        elif not pd.isnull(dataFrame.at[i,'end-planned']) and not pd.isnull(dataFrame.at[i,'start-actual']):
            dataFrame.at[i,'date-diff-days'] = dataFrame.at[i,'end-planned'] - dataFrame.at[i,'start-actual']
            # if less than 365.25 days, round the duration in years to 1
            if dataFrame.at[i,'date-diff-days'].days / 365.25 > 1:
                dataFrame.at[i,'date-diff-years'] = dataFrame.at[i,'date-diff-days'].days / 365.25
            else:
                dataFrame.at[i,'date-diff-years'] = 1
            
        # both end-planned and start-planned are not null - i.e. they are dates
        elif not pd.isnull(dataFrame.at[i,'end-planned']) and not pd.isnull(dataFrame.at[i,'start-planned']):
            dataFrame.at[i,'date-diff-days'] = dataFrame.at[i,'end-actual'] - dataFrame.at[i,'start-actual']
            # if less than 365.25 days, round the duration in years to 1
            if dataFrame.at[i,'date-diff-days'].days / 365.25 > 1:
                dataFrame.at[i,'date-diff-years'] = dataFrame.at[i,'date-diff-days'].days / 365.25
            else:
                dataFrame.at[i,'date-diff-years'] = 1
        
        # otherwise, not enough info. Need to impute later
        else:
            dataFrame.at[i,'date-diff-days'] = pd.NaT
            dataFrame.at[i,'date-diff-years'] = np.NaN

    return dataFrame

sector_data = dateDiff(sector_data)

What does this function do? It evaluates what start and end date data that is available for each row. For data points where this data *is* available, it calculates the difference between the two dates in days, then in years. For data points without this data, these fields are left blank.

Let's delete the *date-diff-days* column, as we won't need it anymore, and then let's check how many blank *date-diff-years* rows exist in our data.

In [22]:
sector_data = sector_data.drop(['date-diff-days'], axis=1)

In [23]:
# NOTE TO SELF
# This code is intended to impute the avg lengths of projects for activities with insufficient date information.
# Fix this to apply to any number of donors at a later date.

# TEMP FIX - drop rows with NAN date-diff-years
sector_data.dropna(axis=0,subset=['date-diff-years'],inplace=True)
sector_data['date-diff-years'].isna().sum()

#data_temp = data[~pd.isnull(data['date-diff-years'])]
#dfid_years_avg = data_temp.groupby(['reporting-org'])['date-diff-years'].mean()[0]  # avg duration of DFID activities
#sida_years_avg = data_temp.groupby(['reporting-org'])['date-diff-years'].mean()[1]  # avg duration of Sida activities
#wbg_years_avg = data_temp.groupby(['reporting-org'])['date-diff-years'].mean()[2]   # avg duration of WBG activities

#for i in data.index:
#    if pd.isnull(data.at[i,'date-diff-years']):
#        if data.at[i,'reporting-org'] == 'Department for International Development':
#            data.at[i,'date-diff-years'] = dfid_years_avg
#        elif data.at[i,'reporting-org'] == 'Swedish International Development Agency (Sida)':
#            data.at[i,'date-diff-years'] = sida_years_avg
#        elif data.at[i,'reporting-org'] == 'World Bank':
#            data.at[i,'date-diff-years'] = wbg_years_avg

0

In [24]:
(sector_data['date-diff-years'] == 0).astype(int).sum(axis=0)

0

Now there are no more rows without a *date-diff-years*, we can use this column to calculate each activity's *annual-Commitment*. 

The annual commitment is the total commitment to the project (*total-Commitment*), divided by its duration in years (*date-diff-years*). While in reality, activities' budgets are not spent uniformly over the course of the activity, this is an easy approximation that will make it easier to compare donors' commitments for certain, specific time frames of interest.

In [25]:
sector_data['annual-Commitment'] = sector_data['total-Commitment'] / sector_data['date-diff-years']

In [26]:
year = 2017
from methods import activityInYear
activityInYear(sector_data, year)
sector_data['in-year-Commitment'] = sector_data['annual-Commitment'] * sector_data['date-in-year']
sector_data['in-year-Commitment'][0:10]

0    0.000000e+00
1    0.000000e+00
2    0.000000e+00
3    0.000000e+00
4    5.379283e+04
5    8.189533e+03
6    0.000000e+00
7    0.000000e+00
8    0.000000e+00
9    1.000685e+06
Name: in-year-Commitment, dtype: float64

In [27]:
sector_data['sector-percentage'] = sector_data['sector-percentage'].fillna(value=0)

In [28]:
sectors_of_interest = ['12110', '12181', '12182', '12191', '12220', '12230', '12240', '12250', '12261', \
                      '12262', '12263', '12281', '12310', '12320', '12330', '12340', '12350', '12382', \
                      '13010', '13020', '13030', '13040', '13081', '15170', '15180', '16063','93013']
sectors_of_interest2 = ['13010', '13020', '13030', '13040', '13081']

def sectorPercentage(target_sectors, activity_sectors, sector_percentages):
    # target_sectors is a list of the sectors which we want included in our results
    results = [False] * len(activity_sectors)
    
    for sector in target_sectors:
        for item in activity_sectors:
            if sector == item:
                results[activity_sectors.index(item)] = True

    sum_percents = 0
    
    for i in range(len(results)):
        if results[i]:
            if sector_percentages[i] == '':
                sum_percents = 1
            else:
                sum_percents += int(sector_percentages[i])
    return sum_percents / 100

def applySectorPercentages(dataFrame, target_sectors):
    
    for i in dataFrame.index:
        dataFrame.at[i, 'target-sectors-percentage'] = sectorPercentage(target_sectors, \
                                                        str(dataFrame.at[i, 'sector-code']).split(';'),
                                                        str(dataFrame.at[i, 'sector-percentage']).split(';'))

In [29]:
applySectorPercentages(sector_data, sectors_of_interest2)
sector_data['relevant-commitment'] = sector_data['in-year-Commitment'] * sector_data['target-sectors-percentage']

In [30]:
sector_data.shape

(9929, 27)

In [31]:
sector_data.to_csv('draft_msd_for_mothers_narrow.csv', index=False)